In [1]:
!pip install -r requirements.txt

In [3]:
import os
import cv2
import sys
import time
import torch
import torchvision
import numpy as np
from torchvision import datasets, transforms

from pytorch_detection import utils
os.chdir('pytorch_detection')
from pytorch_detection import transforms as T
from pytorch_detection import coco_eval
from pytorch_detection import coco_utils
from pytorch_detection import pytorch_utils
from pytorch_detection.engine import train_one_epoch, evaluate
from pytorch_detection.preprocess_model import MarkDataset, get_transform
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from pytorch_utils import *
os.chdir('..')

In [4]:
num_classes = 2
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

### Training part

In [5]:
root = r'Experiment_dataset/' 
dataset = MarkDataset(root, get_transform(train=True))
dataset_test = MarkDataset(root, get_transform(train=False))

In [6]:
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=2, shuffle=True, # num_workers=4,
    collate_fn=utils.collate_fn)

# subroots = ['/Defective_Insulators', '/Normal_Insulators']

indices = torch.randperm(len(dataset)).tolist()
dataset = torch.utils.data.Subset(dataset, indices)

dataset = torch.utils.data.Subset(dataset, indices[:-5])
dataset_test = torch.utils.data.Subset(dataset_test, indices[-5:])

# define training and validation data loaders
# Num_when training models in jupyter notebook The workers parameter can only be 0, otherwise an error will occur, which is commented out here
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=2, shuffle=True, # num_workers=4,
    collate_fn=utils.collate_fn)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=2, shuffle=False, # num_workers=4,
    collate_fn=utils.collate_fn)

In [ ]:
# get the model using our helper function
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False, progress=True, 
                                                          num_classes=num_classes, pretrained_backbone = False,
                                                          trainable_backbone_layers = 5)  

# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]

# Adam or SGD
optimizer = torch.optim.Adam(params, lr=0.0003, weight_decay=0.0005)

# and a learning rate scheduler
# cos learning rate
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=1, T_mult=2)

# let's train it for epochs
num_epochs = 60

for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    # Engine.pyTrain_ofOne_The epoch function takes both images and targets. to(device)
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)

    # update the learning rate
    lr_scheduler.step()

    # evaluate on the test dataset    
    evaluate(model, data_loader_test, device=device)  

    if epoch%10 == 0:
        torch.save(model, r'R_CNN_ResNet_' + str(epoch) + '.pkl')  
    
    print('')
    print('==================================================')
    print('')


/usr/local/lib/python3.6/dist-packages/torchvision/ops/boxes.py:101: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)
  keep = keep.nonzero().squeeze(1)


Epoch: [0]  [ 0/79]  eta: 0:10:18  lr: 0.000004  loss: 3.3298 (3.3298)  loss_classifier: 0.1665 (0.1665)  loss_box_reg: 0.0367 (0.0367)  loss_objectness: 0.7697 (0.7697)  loss_rpn_box_reg: 2.3569 (2.3569)  time: 7.8342  data: 6.8693  max mem: 3598
Epoch: [0]  [10/79]  eta: 0:09:37  lr: 0.000043  loss: 1.5117 (1.7097)  loss_classifier: 0.0895 (0.0984)  loss_box_reg: 0.0252 (0.0289)  loss_objectness: 0.5980 (0.6171)  loss_rpn_box_reg: 0.8543 (0.9653)  time: 8.3740  data: 7.5475  max mem: 4030
Epoch: [0]  [20/79]  eta: 0:08:52  lr: 0.000081  loss: 1.0019 (1.3401)  loss_classifier: 0.0745 (0.0851)  loss_box_reg: 0.0239 (0.0267)  loss_objectness: 0.5554 (0.5783)  loss_rpn_box_reg: 0.3543 (0.6500)  time: 9.0802  data: 8.2649  max mem: 4030
Epoch: [0]  [30/79]  eta: 0:06:30  lr: 0.000119  loss: 0.8900 (1.1904)  loss_classifier: 0.0671 (0.0814)  loss_box_reg: 0.0202 (0.0278)  loss_objectness: 0.5307 (0.5589)  loss_rpn_box_reg: 0.2831 (0.5222)  time: 7.7338  data: 6.9148  max mem: 4030
Epoch: [

### Saving model

In [ ]:
torch.save(model, r'R_CNN_ResNet_46.pkl')
time.sleep(20) 

### Prediction part

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def showbbox_(model, img, thresh):
    # The img entered is a tensor in the 0-1 range        
    model.eval()
    with torch.no_grad():
        '''
        prediction Like:
        [{'boxes': tensor([[1492.6672,  238.4670, 1765.5385,  315.0320],
        [ 887.1390,  256.8106, 1154.6687,  330.2953]], device='cuda:0'), 
        'labels': tensor([1, 1], device='cuda:0'), 
        'scores': tensor([1.0000, 1.0000], device='cuda:0')}]
        '''
        prediction = model([img.to(device)])
        

    img = img.permute(1,2,0)  # C,H,W_H,W,C, for drawing
    img = (img * 255).byte().data.cpu()  # * 255, float to 0-255
    img = np.array(img)  # tensor → ndarray

    img = cv2.cvtColor(np.asarray(img), cv2.COLOR_BGR2RGB)

    for i in range(prediction[0]['boxes'].cpu().shape[0]):
        xmin = round(prediction[0]['boxes'][i][0].item())
        ymin = round(prediction[0]['boxes'][i][1].item())
        xmax = round(prediction[0]['boxes'][i][2].item())
        ymax = round(prediction[0]['boxes'][i][3].item())
        
        label = prediction[0]['labels'][i].item()
        threshold = prediction[0]['scores'][i].item()
        
        if label == 1 and threshold > thresh:
            cv2.rectangle(img, (xmin, ymin), (xmax, ymax), (255, 0, 0), thickness=5)
            cv2.putText(img, 'normal', (xmin, ymin), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0),
                               thickness=2)
        elif label == 0 and threshold > thresh:
            cv2.rectangle(img, (xmin, ymin), (xmax, ymax), (0, 255, 0), thickness=5)
            cv2.putText(img, 'defect', (xmin, ymin), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0),
                               thickness=6)
            
    plt.figure(figsize=(20,15))
    plt.imshow(img)

In [ ]:
model = torch.load(r'R_CNN_ResNet_46.pkl')
model.to(device)

# Check predictions of dataset or dataset_test

img, _ = dataset[34]

# Change thresh to set up different threshold value for predictions

showbbox_(model, img, thresh = 0.7)

In [ ]:
# Different preprocess for .xml files

In [ ]:
# xml_list = os.listdir(root+'/Annotations')
# for xml in xml_list:
#   tree = ET.parse(root + '/Annotations/' + xml)
#   a = tree.find('path')
#   if a.text is None:
#     a.text = xml.split('.')[0] + '.jpg'
#   else:
#     a.text = a.text.split('/')[-1]
#   tree.write(root + '/Annotations/' + xml)

In [ ]:
# import xml.etree.ElementTree as ET
# xml_list = os.listdir(root+'/Annotations')
# for xml in xml_list:
#   tree = ET.parse(root + '/Annotations/' + xml)
#   a = tree.find('path')
#   a_list = [string for string in a.text.split('/') if string != ""]
#   a.text = '/' + '/'.join(a_list)
#   tree.write(root + '/Annotations/' + xml)